# * Import : "AGG_PERF_NEWCO_SNAP"
    WHERE CENTER IN ('Revenue', 'Sales')
    AND NOT REGEXP_LIKE(METRIC_CD, '[0-9]C$|[0-9]H$|[0-9]MCOM$')
    AND TM_KEY_?

## Parameter

In [2]:
import configparser
import datetime as dt
import pandas as pd
import numpy as np
import xlrd
import oracledb

config = configparser.ConfigParser()
config.read('../../my_config.ini')
config.sections()

TDMDBPR_user = config['TDMDBPR']['username']
TDMDBPR_pwd = config['TDMDBPR']['password']
TDMDBPR_db = config['TDMDBPR']['db']
TDMDBPR_host = config['TDMDBPR']['host']
TDMDBPR_port = config['TDMDBPR']['port']

AKPIPRD_user = config['AKPIPRD']['username']
AKPIPRD_pwd = config['AKPIPRD']['password']
AKPIPRD_db = config['AKPIPRD']['db']
AKPIPRD_host = config['AKPIPRD']['host']
AKPIPRD_port = config['AKPIPRD']['port']

curr_dt = dt.datetime.now().date()
str_curr_dt = curr_dt.strftime('%Y%m%d')

In [7]:
# Input parameter

v_param = dict(yr=0, mth_start=202401, mth_end=202406, dt=0)

print(f"\nParameter input...\n\n   -> yr: {v_param['yr']}\n   -> mth_start: {v_param['mth_start']}\n   -> mth_end: {v_param['mth_end']}\n   -> dt: {v_param['dt']}")


Parameter input...

   -> yr: 0
   -> mth_start: 202401
   -> mth_end: 202406
   -> dt: 0


## ETL Process

### Step 1 : AGG_PERF_NEWCO_SNAP
    Delete -> Insert

In [2]:

# v_param = dict(yr=0, mth_start=202401, mth_end=202406, dt=0)

job_start_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
print(f'\nJob Start... {job_start_datetime}')

with open('SQL/Import-AGG_PERF_NEWCO_SNAP.sql', 'r') as sql_file:
    queries = sql_file.read().split(';')
    query = queries[0].strip()
    sql_file.close()

# Connect : TDMDBPR
src_dsn = f'{TDMDBPR_user}/{TDMDBPR_pwd}@{TDMDBPR_host}:{TDMDBPR_port}/{TDMDBPR_db}'
src_conn = oracledb.connect(src_dsn)
print(f'\n{TDMDBPR_db} : Connected')
src_cur = src_conn.cursor()


try:
    print(f'\nProcessing...')
    # print(f"\nProcessing...\n\n   -> data as of (YR: {v_param['yr']}, MTH_Start: {v_param['mth_start']}, MTH_End: {v_param['mth_end']}, DT: {v_param['dt']})")

    # Create Dataframe
    src_cur.execute(query, v_param)
    rows = src_cur.fetchall()
    print(f'\nCreate Dataframe...')
    src_df = pd.DataFrame.from_records(rows, columns=[x[0] for x in src_cur.description])
    print(f'\n   -> src_df : {src_df.shape[0]} rows, {src_df.shape[1]} columns') 

    # Connect : AKPIPRD
    tgt_dsn = f'{AKPIPRD_user}/{AKPIPRD_pwd}@{AKPIPRD_host}:{AKPIPRD_port}/{AKPIPRD_db}'
    tgt_conn = oracledb.connect(tgt_dsn)
    # print(f'\n{AKPIPRD_db} : Connected')
    print(f'\nConnecting... {AKPIPRD_db}')
    tgt_cur = tgt_conn.cursor()

    # # Truncate
    # tgt_cur.execute("TRUNCATE TABLE AUTOKPI.AGG_PERF_NEWCO_SNAP")
    # print(f'\n  -> TRUNCATE : "AGG_PERF_NEWCO_SNAP" : Done !')

    # Delete
    tgt_cur.execute("""
        DELETE AUTOKPI.AGG_PERF_NEWCO_SNAP 
        WHERE TM_KEY_MTH BETWEEN :mth_start AND :mth_end
        """, (v_param['mth_start'], v_param['mth_end']))
    print(f'\n   -> DELETE : "AGG_PERF_NEWCO_SNAP" : Done !')
    
    # Insert
    tgt_cur.executemany("""
        INSERT INTO AGG_PERF_NEWCO_SNAP 
        (TM_KEY_YR, TM_KEY_QTR, TM_KEY_MTH, TM_KEY_WK, TM_KEY_DAY, CENTER, PRODUCT_GRP, COMP_CD, METRIC_GRP, METRIC_CD, METRIC_NAME, SEQ, ACTUAL_AS_OF, AGG_TYPE, RR_IND, GRY_IND, UOM, AREA_TYPE, AREA_CD, AREA_NAME, ACTUAL_SNAP, TARGET_SNAP, BASELINE_SNAP, ACTUAL_AGG, TARGET_AGG, BASELINE_AGG, PPN_TM) 
        VALUES (:1,:2,:3,:4,:5,:6,:7,:8,:9,:10,:11,:12,:13,:14,:15,:16,:17,:18,:19,:20,:21,:22,:23,:24,:25,:26,:27)
        """, rows)
    tgt_cur.close()
    tgt_conn.commit()
    print(f'\n   -> INSERT : "AGG_PERF_NEWCO_SNAP" : Done !')


except oracledb.DatabaseError as e:
    print(f'\nError with Oracle : {e}')


finally:
    src_conn.close()
    print(f'\n{TDMDBPR_db} : Disconnected')

    tgt_conn.close()
    print(f'\n{AKPIPRD_db} : Disconnected')
    
    print(f'\nJob Done !!!')



Job Start... 2024-06-21, 15:04:32

TDMDBPR : Connected

Processing...

   -> data as of (YR: 0, MTH_Start: 202401, MTH_End: 202406, DT: 0)

Create Dataframe...

   -> src_df : 17486172 rows, 27 columns

Connecting... AKPIPRD

   -> DELETE : "AGG_PERF_NEWCO_SNAP" : Done !

   -> INSERT : "AGG_PERF_NEWCO_SNAP" : Done !

TDMDBPR : Disconnected

AKPIPRD : Disconnected

Job Done !!!


In [3]:
src_df.tail()
# src_df.dtypes
# df[['PRODUCT_GRP', 'COMP_CD', 'METRIC_CD', 'METRIC_NAME']].drop_duplicates()

,TM_KEY_YR,TM_KEY_QTR,TM_KEY_MTH,TM_KEY_WK,TM_KEY_DAY,CENTER,PRODUCT_GRP,COMP_CD,METRIC_GRP,METRIC_CD,...,AREA_TYPE,AREA_CD,AREA_NAME,ACTUAL_SNAP,TARGET_SNAP,BASELINE_SNAP,ACTUAL_AGG,TARGET_AGG,BASELINE_AGG,PPN_TM
17486167,2024,20242,202406,2024025,20240620,Sales,TOL,TRUE,Sales,TB3R000600CORP,...,HH,909083,Nakhon Ratchasima,NaN,NaN,NaN,434447.0,508618.885000,NaN,2024-06-21 12:37:25
17486168,2024,20242,202406,2024025,20240620,Sales,TOL,TRUE,Sales,TB3R000600CORP,...,HH,908008,"NTB : Pak Kret, Bang Bua Thong, Sai Noi",NaN,NaN,NaN,321359.0,329636.117333,NaN,2024-06-21 12:37:25
17486169,2024,20242,202406,2024025,20240620,Sales,TOL,TRUE,Sales,TB3R000600CORP,...,H,36X,"Chaiyaphum, Loei",NaN,NaN,NaN,217779.0,219077.676000,NaN,2024-06-21 12:37:25
17486170,2024,20242,202406,2024025,20240620,Sales,TOL,TRUE,Sales,TB3R000600CORP,...,H,005,"NTB : Pak Kret, Bang Bua Thong, Sai Noi",NaN,NaN,NaN,321359.0,329636.117333,NaN,2024-06-21 12:37:25
17486171,2024,20242,202406,2024025,20240620,Sales,TOL,TRUE,Sales,TB3R000600CORP,...,H,67X,"Phetchabun, Phichit",NaN,NaN,NaN,110043.0,128088.430333,NaN,2024-06-21 12:37:25
